In [ ]:
%env AWS_DEFAULT_REGION=us-east-1

In [7]:
import boto3
import pandas as pd
import json

In [3]:
build_permit_text = """
Commencement – Breach of registered restrictive covenant
This permit will not come into effect until the covenant contained in Instrument of Transfer [insert details] in the Register of Titles is removed or varied to avoid a breach of the covenant by this permit
Include in any permit where the grant of the permit would authorise anything that would result in a breach of a registered restrictive covenant Mandatory condition required by Planning and Environment Act 1987 section 62(1)(aa
Expiry – Development
This permit as it relates to development (buildings and works) will expire if one of the following circumstances applies: a) The development is not started within 2 years of the issued date of this permit. b) The development is not completed within 4 years of the issued date of this permit. In accordance with Section 69 of the Planning and Environment Act 1987, an application may be submitted to the responsible authority for an extension of the periods referred to in this condition
Include in all permits for development for buildings and works with appropriate modification to time for starting and completion. Where development is to be undertaken in stages, include additional conditions stating when each stage of development is to be started and completed. See chapter 6.2.1
"""

In [4]:
bedrock_client = boto3.client('bedrock-runtime')

model_id = 'amazon.titan-embed-text-v2:0'

### on-demand invoke embedding model

In [ ]:
sample_input = {
    "inputText": build_permit_text,
    "dimensions": 256,
    "normalize": True
}
body = json.dumps(sample_input)
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept='application/json',
    contentType='application/json'
)

response_body = json.loads(response.get('body').read())
embeddings = response_body['embedding']
print(f"embedding lenght: {len(embeddings)}")
print(f"embedding: {embeddings}")
print(f"response body: {response_body}")

### Synthesize data


In [ ]:
words_count = build_permit_text.split()
print(f"words count: {len(words_count)}")

In [15]:
sample_text_length = 150

assert len(words_count) > sample_text_length

In [ ]:
import random

record_count = 50000
records = []

for i in range(record_count):
    random_words = random.sample(words_count, sample_text_length)
    random_text = ' '.join(random_words)
    records.append(random_text)

print(f"records count: {len(records)}")


output `jsonl` file for batch inference job.

In [29]:
output_file = 'synthetic-data.jsonl'
with open(output_file, 'w') as f:
    for i, record in enumerate(records):
        output = {
            "recordId": str(i), 
            "modelInput": {
                "inputText": record,
                "dimensions": 256,
                "embeddingsByType": "float"
            }
        }
        f.write(json.dumps(output) + '\n')



In [ ]:
account_id = "<<aws account id>>"
bucket_name = f"bedrock-demo-{account_id}-us-east-1"

In [30]:
iam_role_arn = f"arn:aws:iam::{account_id}:role/bedrock-batch-inference-service-role"

input_file_uri = f"s3://{bucket_name}/batch-inference/embedding/building/input/synthetic-data-50k.jsonl"

output_data_uri = f"s3://{bucket_name}/batch-inference/embedding/building/output/"

In [31]:
bedrock = boto3.client('bedrock')



In [33]:
inputDataConfig=({
    "s3InputDataConfig": {
        "s3Uri": input_file_uri
    }
})

outputDataConfig=({
    "s3OutputDataConfig": {
        "s3Uri": output_data_uri
    }
})

response=bedrock.create_model_invocation_job(
    roleArn=iam_role_arn,
    modelId=model_id,
    jobName="building-text-embedding-batch-job-002",
    inputDataConfig=inputDataConfig,
    outputDataConfig=outputDataConfig
)


In [ ]:
print(response)

In [ ]:
from pprint import pprint

res = bedrock.list_model_invocation_jobs()

pprint(res)
